# How complete are AGDC Level 1 scenes compare to USGS?

In [1]:
import geopandas
import itertools
import folium

import os, json, requests, time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import gzip, zipfile

In [2]:
import datacube
from datacube.model import Range
from datetime import datetime
import shapely
from datacube.api import GridWorkflow
from datacube.model import CRS
from shapely.geometry import mapping

In [3]:
def download_file(url, output_dir):
    local_filename = os.path.join(working_dir,url.split('/')[-1])
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [4]:
# make a custom lut to display out heatmap
zeros = [0] * 256
r_range = reversed(range(256))
r_range = list(itertools.chain(r_range,zeros))
g_inc_range = range(256)
g_dec_range = reversed(range(256))
g_range = list(itertools.chain(g_inc_range, g_dec_range))
b_range = range(256)
b_range = list(itertools.chain(zeros, b_range))

max_time = 1440
min_time = 0
rgb_range = []
rgb_range.append(r_range)
rgb_range.append(g_range)
rgb_range.append(b_range)

In [63]:
date_start, date_end = "1986-01-01"  , "2016-10-11" # start date and end date for time range selection

In [5]:
def rgb_combo(rgb_listoflists, period, minutes):
    
    index = int((len(rgb_listoflists[0])/float(period/2)) * minutes)
    
    if index > len(rgb_listoflists[0]):
        index = len(rgb_listoflists[0])-1
    
    red, green, blue = (rgb_range[0][index], rgb_range[1][index],rgb_range[2][index])
    return(red,green,blue)

## Get the latest records from USGS

In [6]:
working_dir = os.path.abspath("/g/data/v10/tmp/")
landsat_csv_list = ["https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_8.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_ETM.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_ETM_SLC_OFF.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-1980-1989.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-1990-1999.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-2000-2009.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-2010-2012.csv.gz"
                   ]

## Get the WRS2 geometry

In [35]:
wrs2_shapefile = 'https://landsat.usgs.gov/sites/default/files/documents/wrs2_descending.zip'
download_file(wrs2_shapefile, working_dir)

'/g/data/v10/tmp/wrs2_descending.zip'

In [7]:
wrs2_shapefile = os.path.join(working_dir, 'wrs2_descending.shp')

In [5]:
wrs2_zip = os.path.join(working_dir, 'wrs2_descending.zip')

!unzip -o $wrs2_zip -d $working_dir

Archive:  /g/data/v10/tmp/wrs2_descending.zip
  inflating: /g/data/v10/tmp/wrs2_descending.dbf  
  inflating: /g/data/v10/tmp/wrs2_descending.prj  
  inflating: /g/data/v10/tmp/wrs2_descending.sbn  
  inflating: /g/data/v10/tmp/wrs2_descending.sbx  
  inflating: /g/data/v10/tmp/wrs2_descending.shp  
  inflating: /g/data/v10/tmp/wrs2_descending.shp.xml  
  inflating: /g/data/v10/tmp/wrs2_descending.shx  


In [8]:
wrs2_gpd = gpd.read_file(wrs2_shapefile)

In [9]:
sat_path = list(range(87,116))
sat_row = list(range(67,91))
print(sat_path)

[87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]


In [10]:
wrs2_path = wrs2_gpd.loc[wrs2_gpd['PATH'].isin(sat_path)]
aoi_wrs2 = wrs2_path.loc[wrs2_path['ROW'].isin(sat_row)]
aoi_wrs2_json = aoi_wrs2.to_crs(epsg='4326').to_json()

In [11]:
scene_map = folium.Map(location=[-30,150],tiles='Mapbox Bright', zoom_start=4)
folium.GeoJson(aoi_wrs2_json,
    style_function=lambda x: {
        
        'weight' : 2,
        'opacity': 0.6,
        
        }).add_to(scene_map)

In [12]:
scene_map

In [13]:
path_row = aoi_wrs2.PR.tolist()

In [14]:
path_row = aoi_wrs2.PATH.map(str) + "_" + aoi_wrs2.ROW.map(str)

In [15]:
for csv in landsat_csv_list:
    download_file(csv, working_dir)

In [16]:
landsat_csv = []
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_8.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_ETM.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_ETM_SLC_OFF.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-1980-1989.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-1990-1999.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-2000-2009.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-2010-2012.csv.gz'))

In [17]:
## TODO read directly to json obj
for csv in landsat_csv:
    inF = gzip.open(csv, 'rb')
    print("Unzipping "+csv)
    outF = open((csv.replace('.gz','')), 'wb')
    outF.write( inF.read() )
    inF.close()
    outF.close()

Unzipping /g/data/v10/tmp/LANDSAT_8.csv.gz
Unzipping /g/data/v10/tmp/LANDSAT_ETM.csv.gz
Unzipping /g/data/v10/tmp/LANDSAT_ETM_SLC_OFF.csv.gz
Unzipping /g/data/v10/tmp/LANDSAT_TM-1980-1989.csv.gz
Unzipping /g/data/v10/tmp/LANDSAT_TM-1990-1999.csv.gz
Unzipping /g/data/v10/tmp/LANDSAT_TM-2000-2009.csv.gz
Unzipping /g/data/v10/tmp/LANDSAT_TM-2010-2012.csv.gz


In [193]:
scene_list = []
time_list = []
landsat_inventory=[os.path.join(working_dir,'LANDSAT_8.csv'),\
                   os.path.join(working_dir,'LANDSAT_ETM.csv'),\
                   os.path.join(working_dir,'LANDSAT_ETM_SLC_OFF.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-1980-1989.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-1990-1999.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-2000-2009.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-2010-2012.csv')
                  ]

In [194]:
for collection in landsat_inventory:
    data_inventory = pd.read_csv(collection , usecols=['acquisitionDate', "sceneID", "path", "row"]) # limit the columns to only the ones we need
    data_inventory['path_row'] = data_inventory["path"].map(str) + "_" + data_inventory["row"].map(str)
    data_inventory['acquisitionDate'] = data_inventory['acquisitionDate'].apply(pd.to_datetime)
    data_inventory = data_inventory.loc[(data_inventory['acquisitionDate'] > pd.tslib.Timestamp(date_start+' 00:00:00'))& (data_inventory['acquisitionDate'] <= pd.tslib.Timestamp(date_end+' 00:00:00'))]
    scene_list.append(data_inventory.loc[data_inventory['path_row'].isin(path_row.tolist())]['sceneID'].tolist())

In [200]:
usgs_sceneid = []
total = 0
for item in scene_list:
    total = len(item) + total
    usgs_sceneid = item + usgs_sceneid
print(len(usgs_sceneid))

410707


In [29]:
scene_list[0][0]

'LC81040832017010LGN00'

# AGDC Level 1 

In [21]:
dc = datacube.Datacube(app='dc-example')

In [22]:
agdc_ls8 = dc.index.datasets.search_eager(product='ls8_level1_scene')
agdc_ls7 = dc.index.datasets.search_eager(product='ls7_level1_scene')
agdc_ls5 = dc.index.datasets.search_eager(product='ls5_level1_scene')

In [23]:
results = agdc_ls5 + agdc_ls7 + agdc_ls8

In [24]:
agdc_sceneid = []
for item in results:
    agdc_sceneid.append(str.split(item.measurements['1']['path'], '/')[1].replace('_B1.TIF', ''))

In [25]:
len(agdc_sceneid)

484523

In [65]:
agdc_sceneid[0]

'LT50940742011307ASA00'

In [54]:
trimmed_usgs = []

In [55]:
for item in usgs_sceneid:
    trimmed_usgs.append(item[:16])

In [56]:
trimmed_usgs[0]

'LT50890862011320'

In [57]:
trimmed_agdc = []
for item in agdc_sceneid:
    trimmed_agdc.append(item[:16])

In [60]:
sceneid_diff = list(set(trimmed_usgs) - set(trimmed_agdc))
len(sceneid_diff)

14591

In [62]:
sceneid_diff[0:10]

['LC80920792016324',
 'LC80960852016288',
 'LT51150692006249',
 'LC80900832016293',
 'LE70960762000284',
 'LC81120842016192',
 'LC81110772016297',
 'LC80960862016336',
 'LE71000782016260',
 'LE70950842017003']

In [179]:
agdc_thing=[]
for list_of_items in [agdc_ls5, agdc_ls7, agdc_ls8]:
    for item in list_of_items:
        thing.append({'path_row': str(item.metadata_doc['image']['satellite_ref_point_start']['x'])\
                      + '_' + str(item.metadata_doc['image']['satellite_ref_point_start']['y']),\
             'day': item.metadata_doc['extent']['center_dt'],\
             'local_path': item.local_path,\
             'sceneID': (str.split(item.measurements['1']['path'], '/')[1].replace('_B1.TIF', ''))[:16]})

In [188]:
agdc_thing = pd.DataFrame(thing)

In [201]:
len(agdc_thing)

484523

In [210]:
agdc_sceneid =[]
agdc_sceneid.append(agdc_thing.loc[agdc_thing['path_row'].isin(path_row.tolist())]['sceneID'].tolist())

In [211]:
len(agdc_sceneid[0])

415611

In [214]:
trimmed_agdc = []
for item in agdc_sceneid[0]:
    trimmed_agdc.append(item[:16])

In [215]:
len(trimmed_agdc)

415611

In [225]:
len(list(set(trimmed_agdc)))

415611

In [224]:
not_in_usgs_archive = list(set(trimmed_agdc) - set(trimmed_usgs))
not_in_agdc_archive = list(set(trimmed_usgs) - set(trimmed_agdc))
print(not_in_usgs_archive[0], len(not_in_usgs_archive), not_in_agdc_archive[0], len(not_in_agdc_archive))

LE71130852012212 14089 LC80920792016324 14591


In [238]:
agdc_days = []
for item in not_in_usgs_archive:
    agdc_days.append(item[9:])

In [239]:
len(list(set(agdc_days)))

5898

In [235]:
usgs_days = []
for item in not_in_agdc_archive:
    usgs_days.append(item[9:])

In [236]:
len(list(set(usgs_days)))

2388

In [249]:
agdc_days_int = list(map(int, agdc_days))

In [257]:
len([i for i in set(agdc_days_int) if i <= 2016286])

5898

In [246]:
usgs_days_int = list(map(int, usgs_days))

In [258]:
len([i for i in set(usgs_days_int) if i <= 2016286])

2298

In [233]:
for item in not_in_agdc_archive:
    if '2016324' in item:
        print(item)

LC80920772016324
LC80920782016324
LC80920792016324
LC80920802016324
LC80920812016324
LC80920822016324
LC80920832016324
LC80920842016324
LC80920852016324
LC80920862016324
LC80920872016324
LC80920882016324
LC80920892016324
LC80920902016324
LC81080672016324
LC81080682016324
LC81080692016324
LC81080702016324
LC81080712016324
LC81080722016324
LC81080732016324
LC81080742016324
LC81080752016324
LC81080762016324
LC81080772016324
LC81080782016324
LC81080792016324
LC81080802016324
LC81080812016324
LC81080822016324
LC81080832016324
LC81080842016324
LC81080852016324
LE70910762016324
LE70910772016324
LE70910782016324
LE70910792016324
LE70910802016324
LE70910812016324
LE70910822016324
LE70910832016324
LE70910842016324
LE70910852016324
LE70910862016324
LE71000672016324
LE71000682016324
LE71000692016324
LE71000702016324
LE71000712016324
LE71000722016324
LE71000732016324
LE71000742016324
LE71000752016324
LE71000762016324
LE71000772016324
LE71000782016324
LE71000792016324
LE71000802016324
LE710008120163

In [240]:
dc.index.datasets.search_eager(product='ls7_level1_scene' '2016-01-01<time<1996-12-31')

Dataset <id=f458197b-bf97-4a7f-a1d6-d1fc49514e3d type=ls7_level1_scene location=/g/data/v10/reprocess/ls7/level1/2016/05/LS7_ETM_SYS_P31_GALPGS01-008_100_082_20160511/ga-metadata.yaml>

In [223]:
for item in not_in_agdc_archive:
    if item in trimmed_agdc:
        print(item)

In [ ]:
from datacube.model import Range
for path in sat_path:
    for row in sat_row:
        path_row_datacube = dc.index.datasets.search_eager(product='ls8_nbar_albers',\
                                                           source_filter=dict(product='ls8_nbar_scene',
                                                                              sat_path = Range(path,path),sat_row = Range(row,row)))
        print(path,row, path_row_datacube)
        #path_row_datacube

In [10]:


time_range =Range(datetime(2013, 1, 1), datetime(2017, 1, 1))
results = dc.index.datasets.search_eager(
    product='ls8_nbar_albers',
    time=time_range,
    source_filter=dict(
        product='ls8_nbar_scene',
        sat_path = Range(87,116),
        sat_row = Range(67,91)
    )
)
results[0]

Dataset <id=9fa161e7-801c-45dd-8b91-51477e83c3b2 type=ls8_nbar_albers location=/g/data/rs0/datacube/002/LS8_OLI_NBAR/18_-37/LS8_OLI_NBAR_3577_18_-37_20160318234306500000.nc>

In [27]:

satellite_map = folium.Map(location=[-30,150],tiles='Mapbox Bright', zoom_start=4)

seamless_query = {
    'time': ('1986-1-1', '1987-6-30')#,

}
product='ls5_nbar_albers'
gw = GridWorkflow(dc.index, product='ls5_nbar_albers')
results = gw.list_cells(product = 'ls5_nbar_albers', 
              **seamless_query)
tile_statistics = {}
features = []
max_list = []

for key in results.keys():
    max_list.append(results[key].shape[0])

max_value = max(max_list)+1

for key in results.keys():
    features.append({'id' : key, 'count': results[key].shape[0], 'geometry' : \
                            mapping(shapely.geometry.polygon.Polygon(results[key].geobox.extent.to_crs(CRS('EPSG:4326')).points)),\
                           'color': '#%02x%02x%02x' % (rgb_combo( rgb_range, max_value,results[key].shape[0])),
                            'type': 'Feature'})

tile_statistics['features']=features
tile_statistics['type']='FeatureCollection'

folium.GeoJson(tile_statistics,
    style_function=lambda x: {
        'color' : x['color'],
        'weight' : 2,
        'opacity': 0.6,
        'fillColor' : x['color']
        }).add_to(satellite_map)    


In [28]:
satellite_map